In [1]:
# Load the CSV file and identify the data types automatically using .option('inferSchema','true')

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': '****',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': '***'
}

configuration_name = 'os_8cbf59ff782d4a5db7683b1dceef1190_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema','true')\
  .load(cos.url('First_Time_Buyer.csv', 'advanceddatasciencecapstone-donotdelete-pr-****'))


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200812135848-0000
KERNEL_ID = 14e04441-ede2-4219-ba63-cce0e69be2be


# Data Cleansing

## 1. Missing values treatment / Imputing

## LSM
LSM is Living Standard Measure and it depends on several factors such as income. The missing values can be imputed using the LSM values with the same income_code and/or province or marital status.

##  province
Province is a categorical variable, as seen previously. The missing province values is considered to be **UNKNOWN**. 

## years_occupation
This column has numerical values. Null values means the idividual does not have any job, so **years_occupation** is set to **ZERO**.

## marital_status
It's a categorical variable. The missing marital status is set to **UNKNOWN**.

In [2]:
# Treating the missing values

# province
df = df.na.fill("Unknown", 'province')

# years_occupation
df = df.na.fill(0, 'years_occupation')

# marital status
df = df.na.fill("Unknown", 'marital_status')


# LSM

# group by 'province' and 'income_code' and find the median of lsm for each group
df.createOrReplaceTempView("dftable")

df_lsm = spark.sql("""
SELECT province, income_code, percentile_approx(lsm, 0.5) AS med_lsm 
FROM dftable 
GROUP BY province,income_code 
ORDER BY province,income_code
""")
df_lsm.show()


# impute missing values with the calculated median
df_lsm.createOrReplaceTempView("df_lsm")

df = spark.sql("""
SELECT d.*, CASE WHEN d.lsm is null THEN l.med_lsm ELSE d.lsm END AS lsm_fixed 
FROM dftable d
LEFT JOIN df_lsm l ON l.province=d.province AND l.income_code=d.income_code
""")


# drop old lsm column
df=df.drop('lsm')

# rename lsm_fixed back to lsm
df=df.withColumnRenamed('lsm_fixed','lsm')

+------------+-----------+-------+
|    province|income_code|med_lsm|
+------------+-----------+-------+
|Eastern Cape|          1|      3|
|Eastern Cape|          2|      3|
|Eastern Cape|          3|      5|
|Eastern Cape|          4|      6|
|Eastern Cape|          5|      7|
|Eastern Cape|          6|      8|
|Eastern Cape|          7|      9|
|Eastern Cape|          8|      9|
|Eastern Cape|          9|     10|
|Eastern Cape|         10|     10|
|Eastern Cape|         11|     10|
|  Free State|          1|      3|
|  Free State|          2|      3|
|  Free State|          3|      5|
|  Free State|          4|      6|
|  Free State|          5|      7|
|  Free State|          6|      8|
|  Free State|          7|      9|
|  Free State|          8|      9|
|  Free State|          9|      9|
+------------+-----------+-------+
only showing top 20 rows



## Remove unnecessary values

As shown before, all the variables look fine except directorship. I am only interested in an individual is an **active director** or a **resigned director** or **not a director**. So all the other values except **DIRECTORSHIP ACTIVE** and **DIRECTORSHIP RESIGNED** must be changed to **NO DIRECTORSHIP**.

In [3]:
# Create a new temporary view of DataFrame
df.createOrReplaceTempView("dftable")

# keep DIRECTORSHIP ACTIVE and DIRECTORSHIP RESIGNED only and change others to NO DIRECTORSHIP
df=spark.sql("""
SELECT *,
CASE WHEN (directorship<>'DIRECTORSHIP ACTIVE' AND directorship<>'DIRECTORSHIP RESIGNED') 
     THEN 'NO DIRECTORSHIP'
     ELSE directorship 
END AS directorship_fixed
FROM dftable
""")

# drop the old directorship column
df=df.drop('directorship')

# rename directorship_fixed column to directorship
df=df.withColumnRenamed('directorship_fixed','directorship')

## 2. Remove correlated variables

As shown in the **Initial data exploration** notebook, **months_risk_score** is perfectly correlated with **months_income**. So the first one is removed from the DataFrame.

In [4]:
df=df.drop('months_risk_score')

## 3. Remove outliers

As shown before, following columns contain outliers.

months_mobile_phone

months_home_phone

months_work_phone

years_occupation


In [5]:
# reset and update outliers

df.createOrReplaceTempView('dftable')

df=spark.sql("""
SELECT *,
CASE WHEN months_mobile_phone>300 THEN 300 ELSE months_mobile_phone END AS months_mobile_phone_fixed,
CASE WHEN months_home_phone>300 THEN 300 ELSE months_home_phone END AS months_home_phone_fixed,
CASE WHEN months_work_phone>300 THEN 300 ELSE months_work_phone END AS months_work_phone_fixed,
CASE WHEN years_occupation>35 THEN 35 ELSE years_occupation END AS years_occupation_fixed
FROM dftable
""")

#drop old columns
dropped_cols = ['months_mobile_phone','months_home_phone','months_work_phone','years_occupation']
df=df.drop(*dropped_cols)

# rename the new fixed columns back to the original names
for col in dropped_cols:
    df=df.withColumnRenamed(col+'_fixed',col)

In [6]:
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.

from project_lib import Project
project = Project(spark.sparkContext, '****', '****')
pc = project.project_context

In [7]:
# save the clean data into a CSV file

project.save_data(file_name = "First_Time_Buyer_Cleaned.csv",data = df.toPandas().to_csv(index=False),overwrite=True)

{'file_name': 'First_Time_Buyer_Cleaned.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'advanceddatasciencecapstone-donotdelete-pr-nr1zlb0dvlh3em',
 'asset_id': '684b0a3d-9af7-4caf-aa52-90ba3fa6408c'}

In [8]:
# save the clean data into Parquet format

df.write.parquet(path=cos.url('First_Time_Buyer_Cleaned.parquet', 'advanceddatasciencecapstone-donotdelete-pr-****'), mode='overwrite')

In [10]:
# save the LSM_Imputer table
df_lsm.write.parquet(path=cos.url('LSM_Imputer.parquet', 'advanceddatasciencecapstone-donotdelete-pr-****'), mode='overwrite')